In [170]:
#Amirreza_Soheili_97463133
import ssl
import numpy as np
import pandas as pd
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from keras import Sequential
from keras.layers import Embedding, Dense, Flatten
from keras.preprocessing import sequence
from keras.datasets import imdb
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.metrics import accuracy_score

In [171]:
def main():
    parameters = set_processing_parameters()
    network_weights = train_imdb_network(parameters)
    analyze_dataset(parameters, network_weights)


def set_processing_parameters():
    parameters = {
        'vocabulary_size': 5000,
        'max_words': 500
    }
    return parameters

In [172]:
def train_imdb_network(parameters):
    (x, y), (_, __) = load_imdb_dataset(parameters['vocabulary_size'])
    x = sequence.pad_sequences(x, maxlen=parameters['max_words'])
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

    classifier = Sequential()
    classifier.add(
        Embedding(
            input_dim=parameters['vocabulary_size'],
            output_dim=64,
            input_length=parameters['max_words']
        )
    )
    classifier.add(Dense(units=64, activation='relu'))
    classifier.add(Dense(units=32, activation='relu'))
    classifier.add(Flatten())
    classifier.add(Dense(units=1, activation='sigmoid'))

    classifier.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )

    classifier.fit(
        x=x_train,
        y=y_train,
        validation_data=(x_test, y_test),
        batch_size=32,
        epochs=5,
        verbose=2
    )

    weights = [
        classifier.layers[i].get_weights()
        for i in range(1, (len(classifier.layers) - 2))
    ]

    return weights

In [176]:
def analyze_dataset(parameters, weights):
    dataset = pd.read_csv('TranslatedDigikalaDataset.csv')
    y = dataset.iloc[:, 1].values
    x = preprocess_text(dataset['Comment'], parameters)

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

    corpus = {
        'x_train': x_train,
        'y_train': y_train,
        'x_test': x_test
    }
    y_prediction = classify_translated_data(parameters, weights, corpus)

    evaluate_classifier(y_test, y_prediction)

In [177]:
dataset.head()

,Comment,Liked
0,A great front-facing phone that's great\r\n,1
1,The appearance of the back may not be like a s...,0
2,Hardware is very powerful and very heavy softw...,1
3,"If you're having trouble with it, it's a good ...",1
4,The screen of this handset is one of the most ...,1


In [178]:
def preprocess_text(corpus, parameters):
    nltk.download('stopwords')
    stopwords = set(nltk.corpus.stopwords.words('english'))
    stemmer = PorterStemmer()
    dataset = pd.read_csv('TranslatedDigikalaDataset.csv')
    tokenizer = word_tokenize(dataset)
    x = []

    for text in corpus:
        tokenized_text = tokenizer.tokenize(text)
        useful_words = [
            stemmer.stem(word)[0]
            for word in tokenized_text
            if word not in stopwords
        ]
        preprocessed_text = ' '.join(useful_words)
        x.append(preprocessed_text)

    cv = CountVectorizer(max_features=parameters['max_words'])
    x = cv.fit_transform(x).toarray()

    return x



In [179]:

def classify_translated_data(parameters, weights, corpus):
    classifier = Sequential()
    classifier.add(
        Embedding(
            input_dim=parameters['vocabulary_size'],
            output_dim=64,
            input_length=parameters['max_words']
        )
    )
    classifier.add(Dense(units=64, activation='relu'))
    classifier.add(Dense(units=32, activation='relu'))

    for i in range(1, len(classifier.layers)):
        classifier.layers[i].set_weights(weights[i-1])
        classifier.layers[i].trainable = False

    classifier.add(Dense(units=32, activation='relu'))
    classifier.add(Flatten())
    classifier.add(Dense(units=1, activation='sigmoid'))

    classifier.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )

    classifier.fit(
        x=corpus['x_train'],
        y=corpus['y_train'],
        batch_size=1,
        epochs=5,
        verbose=2
    )

    y_prediction = (classifier.predict(corpus['x_test']) > 0.5)

    return y_prediction

In [180]:
def evaluate_classifier(y_true, y_prediction):
    print('* Accuracy: {:.2%}'.format(accuracy(y_true, y_prediction)))

In [183]:
if __name__ == '__main__':
    try:
        _create_unverified_https_context = ssl._create_unverified_context
    except AttributeError:
        pass
    else:
        ssl._create_default_https_context = _create_unverified_https_context

In [ ]:
main()